In [23]:
import random
from tqdm import tqdm
from pathlib import Path
import re

In [17]:
import torch
from TTS.api import TTS

# Get device
device = "cuda" if torch.cuda.is_available() else "cpu"

# List available 🐸TTS models
print(TTS().list_models())

# Initialize TTS
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)

# List speakers
print(tts.speakers)

# Run TTS
# ❗ XTTS supports both, but many models allow only one of the `speaker` and
# `speaker_wav` arguments

# TTS with list of amplitude values as output, clone the voice from `speaker_wav`

['tts_models/multilingual/multi-dataset/xtts_v2', 'tts_models/multilingual/multi-dataset/xtts_v1.1', 'tts_models/multilingual/multi-dataset/your_tts', 'tts_models/multilingual/multi-dataset/bark', 'tts_models/bg/cv/vits', 'tts_models/cs/cv/vits', 'tts_models/da/cv/vits', 'tts_models/et/cv/vits', 'tts_models/ga/cv/vits', 'tts_models/en/ek1/tacotron2', 'tts_models/en/ljspeech/tacotron2-DDC', 'tts_models/en/ljspeech/tacotron2-DDC_ph', 'tts_models/en/ljspeech/glow-tts', 'tts_models/en/ljspeech/speedy-speech', 'tts_models/en/ljspeech/tacotron2-DCA', 'tts_models/en/ljspeech/vits', 'tts_models/en/ljspeech/vits--neon', 'tts_models/en/ljspeech/fast_pitch', 'tts_models/en/ljspeech/overflow', 'tts_models/en/ljspeech/neural_hmm', 'tts_models/en/vctk/vits', 'tts_models/en/vctk/fast_pitch', 'tts_models/en/sam/tacotron-DDC', 'tts_models/en/blizzard2013/capacitron-t2-c50', 'tts_models/en/blizzard2013/capacitron-t2-c150_v2', 'tts_models/en/multi-dataset/tortoise-v2', 'tts_models/en/jenny/jenny', 'tts_m

In [18]:
import os
import wave

def calculate_total_wav_length(directory_path):
    """
    Calculate the total duration of all .wav files in the given directory.

    Args:
        directory_path (str): Path to the directory containing .wav files.

    Returns:
        int, int, int: Total duration (hours, minutes, seconds).
    """
    total_duration = 0.0

    for filename in os.listdir(directory_path):
        if filename.endswith(".wav"):
            file_path = os.path.join(directory_path, filename)
            try:
                with wave.open(file_path, 'r') as audio_file:
                    frames = audio_file.getnframes()
                    rate = audio_file.getframerate()
                    duration = frames / float(rate)
                    total_duration += duration
            except Exception as e:
                print(f"Could not process file {filename}: {e}")
    
    # Convert total_duration from seconds to hours, minutes, seconds
    hours, remainder = divmod(total_duration, 3600)
    minutes, seconds = divmod(remainder, 60)

    return int(hours), int(minutes), int(seconds)

def get_highest_index(directory_path):
    max_index = -1
    for filename in os.listdir(directory_path):
        if filename.startswith("text_") and filename.endswith(".txt"):
            try:
                index = int(filename[5:-4])
                max_index = max(max_index, index)
            except ValueError:
                pass
    return max_index

In [4]:
text = '''Коллеги, начнем с анализа объемов добычи, переработки и транспортировки углеводородов за прошедший квартал. Согласно отчету производственного департамента, общий объем добычи нефти составил 12,8 млн тонн, что на 3% превышает плановый показатель. Однако есть отклонения в переработке на двух ключевых НПЗ из-за незапланированных простоев на установках гидрокрекинга. Необходимо доработать подход к техническому обслуживанию оборудования. Кроме того, текущие объемы транспортировки снижены на 1,5% из-за ограничения пропускной способности на одном из магистральных трубопроводов. Предлагаю рассмотреть необходимость перераспределения внешних логистических потоков на ближайший месяц, чтобы минимизировать задержки поставок.

На прошлой неделе мы получили рейтинговый отчет независимой консалтинговой компании о нашем углеродном следе. Уровень выбросов CO2 в перерасчете на баррель добытой нефти за год снизился на 2,3% благодаря введению системы улавливания углекислого газа на одном из месторождений и оптимизации энергопотребления. Однако для достижения внутренних KPI к 2030 году нам нужно масштабировать проекты по внедрению ВИЭ, в частности, ускорить интеграцию солнечной и ветряной генерации на энергетическую инфраструктуру западного кластера. Также важно начать пилотный проект по использованию альтернативных видов топлива для автопарка наших логистических подразделений.

Переходим к анализу состояния производственных мощностей. Трубопроводная система на участке NORD-43 требует срочного технического аудита после обнаружения коррозийного повреждения на магистральном сегменте. Это может повлиять на транспортную нагрузку и безопасность эксплуатации. Помимо этого, обсуждается необходимость перевооружения компрессорных станций: текущие агрегаты уже превышают нормативный срок эксплуатации. Предлагаю подключить департамент главного инженера для формирования предложений по срокам модернизации с учетом бюджета следующего года. Что касается цифровизации, внедрение системы предиктивного мониторинга на участке EAST-12 позволило сократить внеплановые простои оборудования на 15%. Надо рассмотреть возможность расширения данного подхода на другие участки.

Согласно данным геологоразведки текущего года, завершены сейсмические исследования на блоке ZETA-9. Обнаруженные запасы углеводородов оцениваются в 450 млн тонн извлекаемых резервов, что, по нашему мнению, стратегически необходимо для увеличения ресурсной базы компании. Рекомендую ускорить начало буровых работ на этом участке, так как конкурентная активность в регионе усиливается. Кроме того, на блоке ALFA-4 пробурена первая поисковая скважина, текущий дебит составил 320 баррелей в сутки. Это обнадеживающий показатель, который требует вовлечения дополнительного финансирования для дальнейших буровых работ и развитого моделирования резервуаров.

По итогам второго полугодия, выручка компании увеличилась на 8% благодаря благоприятной конъюнктуре на мировом рынке нефти. Тем не менее, есть ряд вопросов по операционным затратам. Удельные затраты на добычу в восточном регионе превышают средний показатель компании на 12% из-за сложных геологических условий. В этой связи важно завершить аудит эксплуатационных расходов и предложить меры по повышению эффективности. Также остается задача снижения затрат на логистику: предлагаю обсудить возможность консолидации транспортных контрактов через единый центр управления закупками. Отдельно стоит отметить необходимость оптимизации капзатрат на модернизацию, о чем упоминалось ранее.

Давайте обсудим возникающие вопросы и распределим ответственных за дальнейшее исполнение решений по каждому из блоков.'''

In [12]:
text2 = "Дорогие коллеги, рад приветствовать вас на сегодняшнем совещании. Наша встреча посвящена трем важным темам: развитию информационных технологий и цифровизации процессов в компании, управлению рисками в международных проектах и участию в тендерах, включая подготовку тендерной документации. В контексте развития информационных технологий и цифровизации процессов мы сосредоточимся на внедрении передовых решений для оптимизации операционных процессов. Исследуем использование облачных технологий для повышения эффективности управления данными и анализирования больших объемов информации в реальном времени. В ближайшем будущем важно будет рассмотреть внедрение искусственного интеллекта и машинного обучения для прогнозирования оборудования и оптимизации использования ресурсов. Вторая тема повестки касается управления рисками в международных проектах. Риски неизбежны, особенно в контексте глобальных операций и нестабильной экономической ситуации. Мы обсудим стратегии, направленные на минимизацию потенциальных угроз, включая диверсификацию поставщиков, проведение регулярного аудита проектов и наличие резервного плана действий на случай форс-мажорных обстоятельств. Кроме того, уделим внимание вопросам геополитических рисков и как они могут повлиять на наши проекты в разных регионах мира. Переходя к третьей теме наших обсуждений, мы коснемся участия в тендерах и подготовки тендерной документации. Нам необходимо усилить нашу конкурентоспособность на этом фронте, обеспечив прозрачность и соответствие нашей документации международным стандартам. Работа на этапе предварительного анализа тендера, а также четкое понимание требований и условий будут ключевыми факторами для успешного участия. Мы также обсудим необходимость использования программных решений для автоматизации этих процессов, что позволит нам ускорить подачу предложений и повысить их качество. Рассмотрим также важность соблюдения правовых норм и наличия консультаций специалистов в области налогового и контрактного права. Спасибо за внимание, и надеюсь на плодотворное обсуждение по каждой из тем сегодня."

In [19]:
dataset_name = "test_dataset_v1"

In [20]:
text_dir = Path(f"./outputs/{dataset_name}/text")
speech_dir = Path(f"./outputs/{dataset_name}/speech")

text_dir.mkdir(parents=True, exist_ok=True)
speech_dir.mkdir(parents=True, exist_ok=True)

In [21]:
index = get_highest_index(text_dir.absolute()) + 1
print(f"Next index: {index}")

Next index: 65


In [24]:

with open('table_dsv.json', 'r') as file:
    for text in tqdm(file.readlines()):
        text = re.sub('\n', " ", text)
        text = re.sub("[!?;]", ".", text)
        text = re.sub(r'\s+', " ", text)

        text = text.split("suffix=")[1].split("text_id")[0].strip()
        
        for sentence in text.split(". "):
            with open(f'outputs/{dataset_name}/text/text_{index}.txt', 'w') as output:
                output.write(sentence)
            tts.tts_to_file(
                text=sentence,
                speaker=random.choice(tts.speakers),
                language="ru",
                file_path=f"outputs/{dataset_name}/speech/speech_{index}.wav"
            )
            index += 1
    

  0%|          | 0/20 [00:00<?, ?it/s]The text length exceeds the character limit of 182 for language 'ru', this might cause truncated audio.
The text length exceeds the character limit of 182 for language 'ru', this might cause truncated audio.
The text length exceeds the character limit of 182 for language 'ru', this might cause truncated audio.
The text length exceeds the character limit of 182 for language 'ru', this might cause truncated audio.
The text length exceeds the character limit of 182 for language 'ru', this might cause truncated audio.
  5%|▌         | 1/20 [01:42<32:30, 102.68s/it]The text length exceeds the character limit of 182 for language 'ru', this might cause truncated audio.
The text length exceeds the character limit of 182 for language 'ru', this might cause truncated audio.
The text length exceeds the character limit of 182 for language 'ru', this might cause truncated audio.
The text length exceeds the character limit of 182 for language 'ru', this might ca

In [25]:
hours, minutes, seconds = calculate_total_wav_length(speech_dir.absolute())
print(f"Total duration of all .wav files: {hours} hours, {minutes} minutes, {seconds} seconds")

Total duration of all .wav files: 0 hours, 59 minutes, 40 seconds


In [11]:
from IPython.display import Audio
import librosa
import os

wav = tts.tts(
  text=text,
  speaker="Craig Gutsy",
  language="ru"
)

display(Audio(wav, rate=24000))

The text length exceeds the character limit of 182 for language 'ru', this might cause truncated audio.
The text length exceeds the character limit of 182 for language 'ru', this might cause truncated audio.


KeyboardInterrupt: 

In [ ]:
# wav = tts.tts(
#   text="Hello world!",
#   language="en"
# )

# TTS to a file, use a preset speaker
tts.tts_to_file(
  text="давайте всех их забаним все равно ничего у нас не работает",
  speaker="Craig Gutsy",
  language="ru",
  file_path="outputs/output_ru.wav"
)

'outputs/output_ru.wav'

In [15]:
import json
import pprint
import re
# Open and read the JSON file
with open('table_dsv.json', 'r') as file:
    test = file.readline()
    test = re.sub('\n', " ", test)
    test = re.sub("[!?;]", ".", test)

print(test.split("suffix=")[1].split("text_id")[0].split(". "))


['Доброе утро, уважаемые коллеги', 'Сегодня у нас на повестке дня четыре ключевых вопроса, которые требуют нашего внимания и обсуждения', 'Первый вопрос касается управления запасами и складским хозяйством', 'Мы сталкиваемся с необходимостью более эффективного управления запасами, чтобы минимизировать издержки и повысить эффективность нашей логистической сети', 'Необходимо рассмотреть внедрение автоматизированных систем учета, более строгую систему анализа потребностей и планирования закупок, а также применение новых технологий, таких как RFID-метки и интеграция с другими системами для обеспечения прозрачности на всех этапах складского учета', 'Второй вопрос повестки дня – это обновление нашей стратегии по миноритарным и крупным партнёрствам', 'В условиях меняющегося рынка крайне важно пересмотреть отношения с нашими сторонними партнерами, возможно, нам стоит рассмотреть серьезные изменения в структуре нашего портфеля партнерств', 'Обсудим необходимость переоценки выгод и рисков от теку

In [11]:
from os import path 
from pydub import AudioSegment 
  
# assign files 
input_file = "AUDIO.mp3"
output_file = "result.wav"
  
# convert mp3 file to wav file 
sound = AudioSegment.from_mp3(input_file) 
sound.export(output_file, format="wav") 

<_io.BufferedRandom name='result.wav'>

/bin/bash: line 1: apt-install: command not found
